<a href="https://colab.research.google.com/github/coderANYA/AI-Course/blob/main/ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/digipodium/Datasets/main/classfication/bank_data.csv')
df

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
num_cols = df.select_dtypes(include='number').columns.tolist()
num_cols.remove('Exited')
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
num_pipe = Pipeline([
    ('scaler', StandardScaler())
])
cat_pipe = Pipeline([
    ('encoder', OrdinalEncoder())
])
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])
preprocessor

In [ ]:
X = df.drop('Exited', axis=1)
y = df['Exited']
X = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Sequential ann**

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
tf.__version__

In [ ]:
X_train.shape

In [ ]:
# classification nn model
model =  tf.keras.models.Sequential()
# 1st hidden layer (+ input layer)
model.add(
    tf.keras.layers.Dense(units=6, activation='relu', input_dim=X_train.shape[1])
)
# 2nd hidden layer
model.add(tf.keras.layers.Dense(units=6, activation='relu'))
# output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# sigmoid activation function for binary classification
# softmax for multi-class classification
# linear for regression

# compile model
model.compile(
    optimizer='adam',        # gradient descent algorithm - Adam is the best
    loss='binary_crossentropy',
)
# bimary_crossentropy for binary classification
# categorical_crossentropy for multi-class classification
# mean_squared_error for regression

# summary
model.summary()

In [ ]:
# config
epochs = 50
batch_size = 8
patience = 10

# early stopping
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=patience,
)

# train model
history = model.fit(
    X_train,                # feature matrix
    y_train,                # target vector
    validation_split=0.2,   # 20% data for validation
    epochs=epochs,          # number of epochs
    batch_size=batch_size,  # batch size
    callbacks=[es],         # early stopping
    verbose=1               # print output
)

In [ ]:
# visualize training history
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train', marker='o', color='r')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# save the model to disk
model.save('model_v1.h5') # creates a HDF5 file 'model_v1.h5'

In [ ]:
# save weights to disk
model.save_weights('model_weights_v1.h5') # creates a HDF5 file 'model_weights_v1.h5'

In [ ]:
# load the model from disk
model = tf.keras.models.load_model('model_v1.h5')

**
# load weights from disk
# declare the model as you did before
# model =  tf.keras.models.Sequential()
# model.add(...)
# model.compile(...)
# model.load_weights('model_weights_v1.h5')
**

In [ ]:
# predict
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) # convert probabilities to binary values
print(y_pred)